In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [59]:
boston_data=load_boston()

cols=boston_data['feature_names']
data=pd.DataFrame(data=boston_data['data'],columns=cols)
features = data.drop(['INDUS', 'AGE'], axis=1)
target=pd.DataFrame(np.log(boston_data['target']),columns=['PRICE'])

In [60]:
features

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [61]:
target

,PRICE
0,3.178054
1,3.072693
2,3.546740
3,3.508556
4,3.589059
...,...
501,3.109061
502,3.025291
503,3.173878
504,3.091042


In [62]:


property_stats=np.ndarray(shape=(1,11))
property_stats=features.mean().values.reshape(1,11)
property_stats

regr=LinearRegression()
regr.fit(features,target)
fitted_vals=regr.predict(features)
MSE=mean_squared_error(target,fitted_vals)
RMSE=np.sqrt(MSE)

In [83]:
def get_log_estimate(noRooms,students_per_class,next_to_river=False,high_confidence=True):
    property_stats[0][4]=noRooms
    property_stats[0][8]=students_per_class
    if (next_to_river):
        property_stats[0][2]=1
    else:
        property_stats[0][2]=0
    log_estimate=regr.predict(property_stats)[0][0]
    
    if (high_confidence):
        upper=log_estimate + 2*RMSE
        lower=log_estimate - 2*RMSE
        interval=95
    else:
        upper=log_estimate + RMSE
        lower=log_estimate - RMSE
        interval=68
    return log_estimate,upper,lower,interval

In [84]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [85]:
def get_dollar_estimate(noRooms,students_per_class,next_to_river=False,high_confidence=True):
    ZILLOW_MEDIAN_PRICE = 583.3
    SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_data.target)
                        
    log_est, upper,lower,interval=get_log_estimate(noRooms,students_per_class,next_to_river, 
                                                   high_confidence)
    dol_est=np.round(np.e**log_est * SCALE_FACTOR * 1000 ,3)
    dol_up=np.round(np.e**upper * SCALE_FACTOR * 1000 ,3)                  
    dol_low=np.round(np.e**lower * SCALE_FACTOR * 1000, 3)
    return dol_est,dol_up,dol_low,interval

In [90]:
get_dollar_estimate(6, 12, True)

(782805.478, 1139002.893, 538000.755, 95)

In [94]:
import valuation as val


In [95]:
val.get_dollar_estimate(6, 12, True)

(array([[782805.478]]), array([[1139002.893]]), array([[538000.755]]), 95)